# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [32]:
import math
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from tqdm import tqdm

In [3]:
df = pd.read_csv('train.csv')
df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [4]:
df = df[:100000]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   item_id               100000 non-null  object 
 1   user_id               100000 non-null  object 
 2   region                100000 non-null  object 
 3   city                  100000 non-null  object 
 4   parent_category_name  100000 non-null  object 
 5   category_name         100000 non-null  object 
 6   param_1               96070 non-null   object 
 7   param_2               56412 non-null   object 
 8   param_3               42510 non-null   object 
 9   title                 100000 non-null  object 
 10  description           92265 non-null   object 
 11  price                 94325 non-null   float64
 12  item_seq_number       100000 non-null  int64  
 13  activation_date       100000 non-null  object 
 14  user_type             100000 non-null  object 
 15  i

Выделим числовые поля из набора данных и выделим целевую переменную

In [14]:
dftrain = df[['price','item_seq_number','image_top_1']]
label = df['deal_probability']

In [10]:
dff = df[['price','item_seq_number','image_top_1','deal_probability']]
train,test = train_test_split(dff, test_size=0.25, random_state=13)
Xtrain,ytrain = train[['price','item_seq_number','image_top_1']], train['deal_probability']
Xtest,ytest = test[['price','item_seq_number','image_top_1']], test['deal_probability']
Xtrain.shape, Xtest.shape

((75000, 3), (25000, 3))

Определим класс для описания датасета

In [12]:
class MyDate(torch.utils.data.Dataset):
   
    def __init__(self, df, target):
        self.df = torch.Tensor(np.array(df))
        self.label = torch.Tensor(np.array(target))

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        row = self.df[idx]
        target = self.label[idx]
        
        return row, target

In [15]:
dataset = MyDate(dftrain,label)

In [16]:
dataset[0]

(tensor([ 400.,    2., 1008.]), tensor(0.1279))

In [17]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.25, random_state=13)


In [21]:
len(train_dataset), len(test_dataset)

(75000, 25000)

Разобьем на батчи и проверим данные с DataFrame

In [9]:
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=6,
                          shuffle=False)

In [10]:
for batch in (train_loader):
    print(batch[0])
    print(batch[1])
    break

tensor([[4.0000e+02, 2.0000e+00, 1.0080e+03],
        [3.0000e+03, 1.9000e+01, 6.9200e+02],
        [4.0000e+03, 9.0000e+00, 3.0320e+03],
        [2.2000e+03, 2.8600e+02, 7.9600e+02],
        [4.0000e+04, 3.0000e+00, 2.2640e+03],
        [1.3000e+03, 9.0000e+00, 7.9600e+02]])
tensor([0.1279, 0.0000, 0.4318, 0.8032, 0.2080, 0.8032])


Сверим с исходными данными

In [11]:
dftrain.head(6)

,price,item_seq_number,image_top_1
0,400.0,2,1008.0
1,3000.0,19,692.0
2,4000.0,9,3032.0
3,2200.0,286,796.0
4,40000.0,3,2264.0
5,1300.0,9,796.0


In [12]:
label[:6]

0    0.12789
1    0.00000
2    0.43177
3    0.80323
4    0.20797
5    0.80323
Name: deal_probability, dtype: float64

Подготовим данные для обучения и валидации


In [22]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=1024,
                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=1024,
                          shuffle=True)

In [37]:
for batch in (train_loader):
    print(batch[0].shape, batch[1].shape )
    break

torch.Size([1024, 3]) torch.Size([1024])


In [55]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
#         self.bn2 = nn.BatchNorm1d(hidden_dim)
#         self.dp = nn.Dropout(0.25)
        self.fc2 = nn.Linear(hidden_dim,2 * hidden_dim)
#         self.bn3 = nn.BatchNorm1d(2*hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, 1)


    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.sigmoid(x)
#         x = self.bn2(x)
#         x = self.dp(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
#         x = self.bn3(x)
        x = self.fc3(x)
        #x = F.softmax(x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.sigmoid(x)
#         x = self.bn2(x)
#         x = self.dp(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
#         x = self.bn3(x)
        x = self.fc3(x)
        return x


net = Net(3, 128)

In [27]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [47]:
criterion = RMSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [56]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [57]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 100 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss))
            running_loss = 0.0

print('Training is finished!')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[1,     1] loss: nan


 10%|████████▎                                                                          | 1/10 [00:00<00:08,  1.07it/s]

[2,     1] loss: nan


 20%|████████████████▌                                                                  | 2/10 [00:01<00:07,  1.09it/s]

...

[9,     1] loss: nan


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:08<00:00,  1.13it/s]

[10,     1] loss: nan


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]

Training is finished!


#### Посмотрите, пожалуйста, что я неправильно сделал. Вроде описал правильно и сеть работает но loss выдает nan.
##### Пробовал убирать нормализацию и dropout, все равно nan

In [81]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        x = self.fc(x)
        return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
#         self.bn1 = nn.BatchNorm1d(input_dim)
#         self.fc1 = Perceptron(input_dim, hidden_dim)
#         self.bn2 = nn.BatchNorm1d(hidden_dim)
#         self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(input_dim, 1)
        
    def forward(self, x):
#         x = self.bn1(x)
#         x = self.fc1(x)
#         x = self.dp(x)
        x = self.fc2(x)
        return x

In [82]:
net = FeedForward(3, 256)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [83]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[1,     1] loss: nan


 10%|████████▎                                                                          | 1/10 [00:00<00:06,  1.33it/s]

[2,     1] loss: nan


 20%|████████████████▌                                                                  | 2/10 [00:01<00:05,  1.36it/s]

...

[9,     1] loss: nan


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:06<00:00,  1.41it/s]

[10,     1] loss: nan


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.40it/s]

Training is finished!
